# UAS Collected Traffic Data Analysis (UAS4T)

The scope of the competition is to evaluate the accuracy of statistical or CI methods in transportation-related
detection problems with specific reference in queue formation in urban arterials. The focus will be on obtaining
results as close as possible to the real data.

As per the requirements of the competition, we tried to develop an algorithm to estimate the maximum length of the queues per lane that
are formed for different approaches of an intersection and roads (to be specified) during the monitoring duration.

This algorithm outputs following components:
i. Maximum length of queue
ii. Lane the maximum length occurred 
iii. Coordinates of the start and end of the maximum queue
iv. Timestamp of the maximum queue occurrence
v. Whether, when and where a spillback is formed

In [1]:
import numpy as np
import cv2
import csv
import json
import os
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from matplotlib import pyplot as plt

In [2]:
#DISP : Display Flag for Vehicle Trajectory,Lanes and Spillback 
DISP = 1

In [3]:
#Color Indications of all the vehicles from the given data
obj_types = {' Car': {'color': (255, 70, 70),
                      'width': 5,
                      'height': 5, },
             ' Medium Vehicle': {'color': (70, 255, 70),
                                 'width': 4,
                                 'height': 4, },
             ' Motorcycle': {'color': (70, 70, 255),
                             'width': 3,
                             'height': 3, },
             ' Heavy Vehicle': {'color': (255, 255, 0),
                                'width': 6,
                                'height': 6, },
             ' Bus': {'color': (70, 100, 255),
                      'width': 6,
                      'height': 6, },
             ' Taxi': {'color': (255, 0, 255),
                       'width': 4,
                       'height': 4, }, }

In [4]:
#Properties of  the trajectory 
traj_props = {}
traj_props['lat_min'] = 1000000
traj_props['lat_max'] = 0
traj_props['lon_min'] = 1000000
traj_props['lon_max'] = 0
traj_props['lon_diff'] = 0
traj_props['lat_diff'] = 0
traj_props['max_time'] = 813
traj_props['min_time'] = 0
traj_props['img_height'] = 0
traj_props['img_width'] = 0
traj_props['scale_trajectory'] = 200000
traj_props['longitude_km'] = 111.2
traj_props['lattitude_km'] = 127.2

In [5]:
#Route information initialization
routes_information={}
routes_names = ['LeofAlexandras_tw_28isOktovriou', 'OktovriouIs28_tw_LeofAlexandras', 'OktovriouIs28_tw_South']
for route in routes_names:
    route_information={}
    route_information['direction']=0
    route_information['orientation_range'] = []
    route_information['max_queue']={}
    route_information['max_queue']['length'] = 0
    route_information['max_queue']['time'] = 0.00
    route_information['max_queue']['points'] = []
    route_information['max_queue']['n_vehicles'] = []
    routes_information[route]= route_information

routes_information['LeofAlexandras_tw_28isOktovriou']['direction'] = 0
routes_information['OktovriouIs28_tw_LeofAlexandras']['direction'] = 225
routes_information['OktovriouIs28_tw_South']['direction'] = 90

routes_information['LeofAlexandras_tw_28isOktovriou']['orientation_range'] = [337.5, 22.5]
routes_information['OktovriouIs28_tw_LeofAlexandras']['orientation_range'] = [202.5, 247.5]
routes_information['OktovriouIs28_tw_South']['orientation_range'] = [67.5, 112.5]
routes_information['LeofAlexandras_tw_28isOktovriou']['lane_axis'] = [0, 0]  # 0 col  1  row
routes_information['OktovriouIs28_tw_LeofAlexandras']['lane_axis'] = [0, 1]
routes_information['OktovriouIs28_tw_South']['lane_axis'] = [1, 1]

### Locating the Lane Areas

Location of the lane areas is determined by anchor drawing concept. Proposed approach is modified / improved version of edge drawing algorithm and called as anchor drawing. In anchor drawing, we will draw a line passing through the maximum peak pixels. We get continuous line in anchor drawing compared to edge drawing.

In [6]:
class AnchorDrawing:
    """
    A class to draw the anchor line to locate the lane.
    ...
    Attributes
    ----------
    Methods
    -------
    moveUp_(x, y):
        Compute next peak value towards up direction 
    moveDown_(x, y):
        Compute next peak value towards down direction
    moveRight_(x,y):
        Compute next peak value towards right direction
    moveLeft_(x, y):
        Compute next peak value towards left direction
    moveon_peak_points(x, y, direct_next):
        Compute the next peaks
    compute_anchors(image):
        Computing initial peak or anchor points list
    Draw(image):
        Entry point of the class
    """
    # initiation
    def __init__(self):
        self.anchorThreshold_ = 50
        # dimension of image
        self.height = 0
        self.width = 0
        self.Degree = np.array([])
        self.anchor_image = np.array([])
        self.Img = np.array([])

        self.horizontal_move = 1
        self.vertical_move = -1
        self.left_move = -1
        self.right_move = 1
        self.up_move = -1
        self.down_move = 1


    def moveUp_(self, x, y):
        '''
        Compute next peak value towards up direction 
            Input Parameters:
                    x: Row index  
                    y: Column index
            Output Parameters:
                    list_points: Segment of peak points 
                    direct_next: Search direction of left side similart to right, up and down
        '''
        list_points = []  # array to store peak points
        direct_next = None  # search direction of left side similart to right, up and down
        while x > 0 and self.Img[x, y] > 0 and not self.anchor_image[x, y]:
            next_y = [max(0, y - 1), y, min(self.width - 1, y + 1)]  # search in a valid area
            list_points.append((x, y))  # extend line segments
            if self.Degree[x, y] == self.vertical_move:
                self.anchor_image[x, y] = True  # mark as anchor peak
                y_last = y  # record parent pixel
                x, y = x - 1, next_y[np.argmax(self.Img[x - 1, next_y])]  # walk to next pixel with max gradient
            else:
                direct_next = y - y_last  # change direction to continue search
                break  # stop and proceed to next search
        return list_points, direct_next

    def moveDown_(self, x, y):
        '''
        Compute next peak value towards down direction 
            Input Parameters:
                    x: Row index 
                    y: Column index
            Output Parameters:
                    list_points: Segment of peak points
                    direct_next: Search direction of left side similart to right, up and down
        '''
        list_points = []
        direct_next = None
        while x < self.height - 1 and self.Img[x, y] > 0 and not self.anchor_image[x, y]:
            next_y = [max(0, y - 1), y, min(self.width - 1, y + 1)]
            list_points.append((x, y))
            if self.Degree[x, y] == self.vertical_move:
                self.anchor_image[x, y] = True
                y_last = y
                x, y = x + 1, next_y[np.argmax(self.Img[x + 1, next_y])]
            else:
                direct_next = y - y_last
                break
        return list_points, direct_next

    def moveRight_(self, x, y):
        '''
        Compute next peak value towards right direction 
            Input Parameters:
                    x: Row index
                    y: Column index
            Output Parameters:
                    list_points: Segment of peak points 
                    direct_next: Search direction of left side similart to right, up and down
        '''
        list_points = []
        direct_next = None
        while y < self.width - 1 and self.Img[x, y] > 0 and not self.anchor_image[x, y]:
            next_x = [max(0, x - 1), x, min(self.height - 1, x + 1)]
            list_points.append((x, y))
            if self.Degree[x, y] == self.horizontal_move:
                self.anchor_image[x, y] = True
                x_last = x
                x, y = next_x[np.argmax(self.Img[next_x, y + 1])], y + 1
            else:
                direct_next = x - x_last
                break
        return list_points, direct_next

    def moveLeft_(self, x, y):
        '''
        Compute next peak value towards left direction 
            Input Parameters:
                    x: Row index 
                    y: Column index
            Output Parameters:
                    list_points: Segment of peak points 
                    direct_next: Search direction of left side similart to right, up and down
        '''
        list_points = []
        direct_next = None
        while y > 0 and self.Img[x, y] > 0 and not self.anchor_image[x, y]:
            next_x = [max(0, x - 1), x, min(self.height - 1, x + 1)]
            list_points.append((x, y))
            if self.Degree[x, y] == self.horizontal_move:
                self.anchor_image[x, y] = True
                x_last = x
                x, y = next_x[np.argmax(self.Img[next_x, y - 1])], y - 1
            else:
                direct_next = x - x_last
                break
        return list_points, direct_next

    def moveon_peak_points(self, x, y, direct_next):
        '''
        Returns the 
            Input Parameters:
                    x: Row index
                    y: Column index
                    direct_next: Search direction of left side similart to right, up and down
            Output Parameters:
                    list_points: Segment of peak points
        '''
        list_points = [(x, y)]
        while direct_next is not None:
            x, y = list_points[-1][0], list_points[-1][1]
            # if the last point is towords horizontal, search horizontally
            if self.Degree[x, y] == self.horizontal_move:
                # get points sequence
                if direct_next == self.left_move:
                    s, direct_next = self.moveLeft_(x, y)
                elif direct_next == self.right_move:
                    s, direct_next = self.moveRight_(x, y)
                else:
                    break
            elif self.Degree[x, y] == self.vertical_move:  # search vertically

                if direct_next == self.up_move:
                    s, direct_next = self.moveUp_(x, y)
                elif direct_next == self.down_move:
                    s, direct_next = self.moveDown_(x, y)
                else:
                    break
            else:  # invalid point found
                break
            if len(s) > 1:
                list_points.extend(s[1:])
        return list_points

    # find list of anchors
    def compute_anchors(self, image):
        '''
        Computing initial peak or anchor points list 
            Input Parameters:
                    image: Input accumulated image with vehicle trajectories
            Output Parameters:
                    anchor_list: List of anchor points
        '''
        # detect the anchor points
        anchor_list = []
        self.Degree = np.zeros(image.shape, np.float64)
        for row in range(1, self.height - 1):
            for col in range(1, self.width - 1):
                if (image[row, col] > self.anchorThreshold_):
                    if ((image[row - 1, col] < image[row, col] and image[row + 1, col] < image[row, col]) or \
                            (image[row, col - 1] < image[row, col] and image[row, col + 1] < image[row, col]) or\
                            (image[row - 1, col - 1] < image[row, col] and image[row + 1, col + 1] < image[row, col]) or\
                            (image[row - 1, col + 1] < image[row, col] and image[row + 1, col - 1] < image[row, col])):
                        anchor_list.append((row, col))
                    ysum = int(image[row-1, col])+image[row+1, col] + image[row-1, col-1]+image[row+1, col-1] + image[row-1, col+1]+image[row+1, col+1]
                    xsum = int(image[row, col-1])+image[row, col+1] + image[row-1, col-1]+image[row-1, col+1] + image[row+1, col-1]+image[row+1, col+1]

                    if (ysum  >  xsum):
                        self.Degree[row, col] = -1
                    else:
                        self.Degree[row, col] = 1

        return anchor_list

    def Draw(self, image):
        '''
        Entry point of the class
            Input Parameters:
                    image: Input accumulated image with vehicle trajectories
            Output Parameters:
                    anchor_line: List of anchor points
        '''
        self.height = image.shape[0]
        self.width = image.shape[1]
        self.Img = image.copy()
        # compute anchor points list
        anchor_list = self.compute_anchors(image)
        anchor_line = []
        self.anchor_image = np.zeros(self.Img.shape, dtype=bool)
        for anchor in anchor_list:
            if not self.anchor_image[anchor]:  # if not mark as anchor peak
                # serch for next peak point in direction 1
                point_list1 = self.moveon_peak_points(anchor[0], anchor[1], 1)
                self.anchor_image[anchor] = False
                # serch for next peak point in direction -1
                point_list2 = self.moveon_peak_points(anchor[0], anchor[1], -1)
                # concat two point lists
                if len(point_list1[::-1] + point_list2) > 0:
                    anchor_line.append(point_list1[::-1] + point_list2[1:])
        return anchor_line

In [7]:
def readdata(file_name):
    '''
    Reading the data from given input csv 
            Input Parameters:
                    file_name: csv filename/path
            Output Parameters:
                    data: Dictionary of all trajectory points along with vehicle information
    '''
    csv_file = open(file_name, 'r')

    lines = csv_file.readlines()
    num_lines = len(lines)

    object_list = []
    for row in range(1, num_lines):
        # all_lines.append(row)
        line = lines[row]
        line_parts = line.split(';')
        object_prop = {}
        object_prop['trajectory'] = {}
        object_prop['trajectory']['lat'] = []
        object_prop['trajectory']['lon'] = []
        object_prop['trajectory']['speed'] = []
        object_prop['trajectory']['lon_acc'] = []
        object_prop['trajectory']['lat_acc'] = []
        object_prop['trajectory']['time'] = []
        object_prop['trajectory']['x'] = []
        object_prop['trajectory']['y'] = []

        object_prop['track_id'] = int(line_parts[0])
        object_prop['type'] = line_parts[1]
        object_prop['traveled_d'] = float(line_parts[2])
        object_prop['avg_speed'] = float(line_parts[3])
        for step in range(4, len(line_parts) - 6, 6):
            latitude = float(line_parts[step])
            longitude = float(line_parts[step + 1])
            speed_v = float(line_parts[step + 2])
            latitude_acc = float(line_parts[step + 3])
            longitude_acc = float(line_parts[step + 4])
            time_stamp = float(line_parts[step + 5])
            object_prop['trajectory']['lat'].append(latitude)
            object_prop['trajectory']['lon'].append(longitude)
            object_prop['trajectory']['speed'].append(speed_v)
            object_prop['trajectory']['lon_acc'].append(longitude_acc)
            object_prop['trajectory']['lat_acc'].append(latitude_acc)
            object_prop['trajectory']['time'].append(time_stamp)

            if (traj_props['lon_max'] < longitude):
                traj_props['lon_max'] = longitude
            if (traj_props['lat_max'] < latitude):
                traj_props['lat_max'] = latitude
            if (traj_props['lon_min'] > longitude):
                traj_props['lon_min'] = longitude
            if (traj_props['lat_min'] > latitude):
                traj_props['lat_min'] = latitude
            if (traj_props['min_time'] > time_stamp):
                traj_props['min_time'] = time_stamp
            if (traj_props['max_time'] < time_stamp):
                traj_props['max_time'] = time_stamp

        object_list.append(object_prop)

    traj_props['lon_diff'] = traj_props['lon_max'] - traj_props['lon_min']
    traj_props['lat_diff'] = traj_props['lat_max'] - traj_props['lat_min']

    traj_props['img_height'] = int(round(traj_props['lat_diff'] * traj_props['scale_trajectory']))
    traj_props['img_width'] = int(round(traj_props['lon_diff'] * traj_props['scale_trajectory']))


    data = {}
    data['object_list'] = object_list
    data['traj_props'] = traj_props

    return data

In [8]:
def get_line(point1, point2):
    '''
    Computes list of line points between two points  
            Input Parameters:
                    point1:(Col, Row)
                    point2:(Col, Row)
            Output Parameters:
                    points: list of line points from point1 to point2
    '''
    points = []
    issteep = abs(point2[1] - point1[1]) > abs(point2[0] - point1[0])
    if issteep:
        point1[0], point1[1] = point1[1], point1[0]
        point2[0], point2[1] = point2[1], point2[0]
    rev = False
    if point1[0] > point2[0]:
        point1[0], point2[0] = point2[0], point1[0]
        point1[1], point2[1] = point2[1], point1[1]
        rev = True
    deltax = point2[0] - point1[0]
    deltay = abs(point2[1] - point1[1])
    error = int(deltax / 2)
    y = point1[1]

    if point1[1] < point2[1]:
        ystep = 1
    else:
        ystep = -1
    for x in range(point1[0], point2[0] + 1):
        if issteep:
            points.append((y, x))
        else:
            points.append((x, y))
        error -= deltay
        if error < 0:
            y += ystep
            error += deltax
    # Reverse the list if the coordinates were reversed
    if rev:
        points.reverse()
    return points

In [9]:
def find_ang(p1, p2):
    '''
    Computes the angle of line with respect to horizontal axis 
            Input Parameters:
                    p1: (Col, Row)
                    p2: (Col, Row)
            Output Parameters:
                    angle_in_degrees: Angle of the line(0-360)
    '''
    #  ********90********
    #  **45********135**
    #  0**************180
    #  **-45*******-135*
    #  *******-90*******
    angle_in_degrees = np.arctan2(p1[1] - p2[1], p1[0] - p2[0]) * 180 / np.pi
    if (angle_in_degrees < 0):
        angle_in_degrees += 360
    return angle_in_degrees

In [10]:
def normalizeimg(accum):
    '''
    Linear normalization of  accumated array with vehicle trajectories 
            Input Parameters:
                    accum: accumulated array
            Output Parameters:
                    accum: normalized accumulated array(0-255)
    '''
    min = 0
    try:
        min = accum[accum != 0].min()
    except:
        print()
    if (min != 0):
        accum[accum != 0] = (accum[accum != 0] - min) * (255 / float(accum.max() - min + 0.000000001))
    else:
        accum = (accum - accum.min()) * (255 / float(accum.max() - accum.min() + 0.000000001))
    return accum

In [11]:
def getLanePoints(img_side,direction=0, rRad = 3 ,n_lanes = 3, start_point = None):
    '''
    Computes lane areas using anchor drawing algorithm 
            Input Parameters:
                    img_side: Normalized accumulated array
                    direction: Direction of the route
                    rRad: Road radius approximate value
                    n_lanes: Number of lanes default
                    start_point: Starting point of lane (Col, Row)
            Output Parameters:
                    max_length_lanes: Returns lane information of the route with lane points, polygons
    '''
    img_height = img_side.shape[0]
    img_width = img_side.shape[1]
    ad = AnchorDrawing()
    edges = ad.Draw(img_side)
    len_indices = []
    [len_indices.append(len(item)) for item in edges]
    areas_index = np.argsort(np.array(len_indices))

    max_length_lanes = []
    for lNum in range(-1, -(n_lanes+1), -1):
        lane_info={}
        lpoints = edges[areas_index[lNum]]

        if direction ==0:
            if (lpoints[0][1] > lpoints[-1][1]):
                lpoints.reverse()
        elif direction == 90:
            if (lpoints[0][0] > lpoints[-1][0]):
                lpoints.reverse()
            if start_point:
                row_vector= [p[0] for p in lpoints]
                if start_point[1] in row_vector:
                    idx = row_vector.index(start_point[1])
                    lpoints = lpoints[idx:]
        elif direction == 225:
            if (lpoints[0][1] < lpoints[-1][1]):
                lpoints.reverse()

        lane_info['lane_points'] = [(p[1], p[0]) for p in lpoints]

        line1=[]
        line2=[]
        ang = 0
        # Compute polygon area using lane points and road radius
        for pnum in range(0, len(lpoints)):

            if (pnum < len(lpoints)-1):
                ang = find_ang((lpoints[pnum][1],lpoints[pnum][0]), (lpoints[pnum+1][1], lpoints[pnum+1][0]))
            if (ang == 45.0 ):# Push in the form of  x,y
                line1.append(( max(0,lpoints[pnum][1]-rRad),min(img_height, lpoints[pnum][0]+rRad)))
                line2.append((min(img_width, lpoints[pnum][1] + rRad) , max(0, lpoints[pnum][0] - rRad)) )
            elif(ang == 225.0):
                line1.append((min(img_width, lpoints[pnum][1] + rRad), max(0, lpoints[pnum][0] - rRad)) )
                line2.append((max(0, lpoints[pnum][1] - rRad), min(img_height, lpoints[pnum][0] + rRad)))
            elif(ang == 0.0):
                line1.append((lpoints[pnum][1] , min(img_height, lpoints[pnum][0] + rRad)))
                line2.append((lpoints[pnum][1] , max(0, lpoints[pnum][0] - rRad)))
            elif (ang == 180.0):
                line1.append((lpoints[pnum][1], max(0, lpoints[pnum][0] - rRad)) )
                line2.append((lpoints[pnum][1], min(img_height, lpoints[pnum][0] + rRad)))
            elif(ang == 135.0):
                line1.append((max(0, lpoints[pnum][1] - rRad), max(0, lpoints[pnum][0] - rRad)))
                line2.append((min(img_width, lpoints[pnum][1] + rRad), min(img_height, lpoints[pnum][0] + rRad)))
            elif (ang == 315.0):
                line1.append((min(img_width, lpoints[pnum][1] + rRad), min(img_height, lpoints[pnum][0] + rRad)) )
                line2.append((max(0, lpoints[pnum][1] - rRad), max(0, lpoints[pnum][0] - rRad)))
            elif(ang == 90.0):
                line1.append((max(0, lpoints[pnum][1] - rRad), lpoints[pnum][0]))
                line2.append((min(img_width, lpoints[pnum][1] + rRad), lpoints[pnum][0]))
            elif (ang == 270.0):
                line1.append((min(img_width, lpoints[pnum][1] + rRad), lpoints[pnum][0]) )
                line2.append((max(0, lpoints[pnum][1] - rRad), lpoints[pnum][0]))

        line2.reverse()
        poly_lane = line1+line2
        poly_lane.append(line1[0])
        polygon = Polygon(poly_lane)
        lane_info['poly'] = polygon
        lane_info['vertices'] = poly_lane
        max_length_lanes.append(lane_info)
    return max_length_lanes

In [12]:
def get_accumulator(trajectory_data, padd, routes_information):
    '''
    Computes accumulated array with all the vehicle trajectories 
            Input Parameters:
                    trajectory_data: Data contains vehicle trajectory points and properties
                    padd: Padding value for both rows and columns
                    routes_information: Contains route direction, orientation ranges
            Output Parameters:
                    accum: Dictionary with accumulated array for all the routes
    '''
    adj_ang = -15  # Angle adjustment to keep routes aligned with clear direction
    point_step = 30  # point step to get smooth line points
    ang_step = 2  # angle step to find angle/direction of vehicle in trajectory
    accum_weights = [[1, 1, 1],
                     [1, 2, 1],
                     [1, 1, 1]]  # accumulator weights array  highlights hehicle path
    accum_weights = np.array(accum_weights, np.uint8) * 2


    object_list = trajectory_data['object_list']
    traj_props = trajectory_data['traj_props']
    padd_x = padd
    padd_y = padd
    acc_w =traj_props['img_width']+ padd+padd+1
    acc_h = traj_props['img_height']+ padd+padd+1
    routes_names = list(routes_information.keys())
    accum = {}
    for route_name in routes_names:
        accum[route_name] = np.zeros([acc_h, acc_w], np.uint32)


    num_obj = len(object_list)

    for id_num in range(0, num_obj):
        length = len(object_list[id_num]['trajectory']['time'])
        for idx in range(0, length):
            data['object_list'][id_num]['trajectory']['x'].append(
                data['traj_props']['scale_trajectory'] * (object_list[id_num]['trajectory']['lon'][idx] - traj_props['lon_min']))
            data['object_list'][id_num]['trajectory']['y'].append(
                (data['traj_props']['scale_trajectory'] * (object_list[id_num]['trajectory']['lat'][idx] - traj_props['lat_min'])))

        comp_line_points = []
        xx1 = round(data['object_list'][id_num]['trajectory']['x'][0]) + padd_x
        yy1 = round(data['object_list'][id_num]['trajectory']['y'][0]) + padd_y
        start_point = min(point_step, length)
        for idx in range(start_point, length - point_step, point_step):
            xx2 = round(data['object_list'][id_num]['trajectory']['x'][idx]) + padd_x
            yy2 = round(data['object_list'][id_num]['trajectory']['y'][idx]) + padd_y
            points = get_line([xx1, yy1], [xx2, yy2])
            [comp_line_points.append(p) for p in points if p not in comp_line_points]
            xx1 = xx2
            yy1 = yy2

        # Compute angle of each point in line
        list_ang = []
        length = len(comp_line_points)
        for idx in range(0, length):
            ang = 0
            if (length - ang_step > idx):
                ang = find_ang(comp_line_points[idx], comp_line_points[idx + ang_step])
            list_ang.append(ang)


        for idx in range(ang_step, length):
            for route_name in routes_names:
                if (routes_information[route_name]['orientation_range'][1] < routes_information[route_name]['orientation_range'][0] ):
                    if (list_ang[idx] <= routes_information[route_name]['orientation_range'][1] + adj_ang or list_ang[idx] > routes_information[route_name]['orientation_range'][0] + adj_ang):  # WW
                        accum[route_name][comp_line_points[idx][1] - padd:comp_line_points[idx][1] + padd + 1,
                        comp_line_points[idx][0] - padd: comp_line_points[idx][0] + padd + 1] += accum_weights  # vec_acc_filter[6]
                else:
                    if (list_ang[idx] <= routes_information[route_name]['orientation_range'][1] + adj_ang and list_ang[idx] > routes_information[route_name]['orientation_range'][0] + adj_ang):  # WW
                        accum[route_name][comp_line_points[idx][1] - padd:comp_line_points[idx][1] + padd + 1,
                        comp_line_points[idx][0] - padd: comp_line_points[idx][0] + padd + 1] += accum_weights  # vec_acc_filter[6]


    return accum

In [13]:
def swap_data(lane_data, vehi_num1, vehi_num2):
    '''
    Swaps vehicle information in lanedata  
            Input Parameters:
                    lane_data: Contains vehciles information(points, ids, speed)
                    vehi_num1: Vehicle index in lane
                    vehi_num2: Vehicle index in lane
            Output Parameters:                
    '''
    # Swap vehicle properties
    temp = lane_data['ID'][vehi_num1]
    lane_data['ID'][vehi_num1] = lane_data['ID'][vehi_num2]
    lane_data['ID'][vehi_num2] = temp
    temp = lane_data['x'][vehi_num1]
    lane_data['x'][vehi_num1] = lane_data['x'][vehi_num2]
    lane_data['x'][vehi_num2] = temp
    temp = lane_data['y'][vehi_num1]
    lane_data['y'][vehi_num1] = lane_data['y'][vehi_num2]
    lane_data['y'][vehi_num2] = temp
    temp = lane_data['speed'][vehi_num1]
    lane_data['speed'][vehi_num1] = lane_data['speed'][vehi_num2]
    lane_data['speed'][vehi_num2] = temp

In [14]:
def sort_ids(routes_data , lane_name, _direction):
    '''
    Sorts vehicle IDs using position and direction of lane 
            Input Parameters:
                    routes_data: Contains route information along with vehicle data
                    lane_name: Route name 
                    _direction: Direction of the route
            Output Parameters:
                    
    '''
    # Sort vehicle IDs using position and direction of lane
    for lane_num in range(0, len(routes_data[lane_name])):

        for vehi_num1 in range(0 , len(routes_data[lane_name][lane_num]['ID'])):
            for vehi_num2 in range(vehi_num1+1 , len(routes_data[lane_name][lane_num]['ID'])):

                if _direction == 0:
                    if(routes_data[lane_name][lane_num]['x'][vehi_num1] > routes_data[lane_name][lane_num]['x'][vehi_num2]):
                        swap_data(routes_data[lane_name][lane_num], vehi_num1, vehi_num2)
                if _direction == 90:
                    if(routes_data[lane_name][lane_num]['y'][vehi_num1] > routes_data[lane_name][lane_num]['y'][vehi_num2]):
                        swap_data(routes_data[lane_name][lane_num], vehi_num1, vehi_num2)
                if _direction == 225:
                    if (routes_data[lane_name][lane_num]['x'][vehi_num1] < routes_data[lane_name][lane_num]['x'][vehi_num2]):
                        swap_data(routes_data[lane_name][lane_num], vehi_num1, vehi_num2)

In [15]:
def finddist(p1, p2):
    '''
    Finds distance between two points 
            Input Parameters:
                    p1: (Col, Row)
                    p2: (Col, Row)
            Output Parameters:
    '''
    return ((p1[0]-p2[0])*(p1[0]-p2[0]) + (p1[1]-p2[1])*(p1[1]-p2[1]) ) ** 0.5

In [16]:
def find_queue_info(routes_data, laneInformation, temporal_data, route_name, axis_vec ):
    '''
    Computes vehicles inqueue with respect to lane number in the route,
    Estimates spillback positions according to queue information  
            Input Parameters:
                    routes_data: Contains route information along with vehicle data
                    laneInformation: Lane area information
                    temporal_data: Previous timeframe routes data
                    route_name: Name of the route
                    axis_vec: Axis of start point, end point of the lane
            Output Parameters:
                    queue_spillback_data: Returns array of queues, spillback points with respect to lane, routes 
    '''
    n_lanes = len(routes_data[route_name])
    speed_threshold = 7  # vehicle speed threshold (if less consider as queue vehicle)
    gap_bw_vehicles = 2 # maximum number of vehicles non stationary of vehicles
    max_dist_gap = 30 # max distance gap in queue
    threshold_spill_to_lane_startpoint = 20  # if spill happen at start position of lane avoide it
    num_temporal_frams = 3 # number of temporal frames to be considered

    out_queue_info = []
    for l_num in range(0, n_lanes):
        out_queue_info.append([])

    spillback_data = {}
    spillback_data['exist'] = False

    queue_data = {}
    queue_data['exist'] = False

    new_ids_in_current_frame_q = []
    route_queue_indices = []
    for l_num in range(0, n_lanes):  # iterate through lane numbers
        list_vehicle_details = {}
        list_vehicle_details['point'] = []
        list_vehicle_details['index'] = []
        # find stationary vehicles in lane
        for v_num in range(0, len(routes_data[route_name][l_num]['ID'])): # iterate through vehicles in lane
            if (routes_data[route_name][l_num]['speed'][v_num] < speed_threshold ):
                list_vehicle_details['point'].append( [ routes_data[route_name][l_num]['x'][v_num] , routes_data[route_name][l_num]['y'][v_num]])
                list_vehicle_details['index'].append(v_num)
        _queue_list = []
        _indices_list = []
        _queue = []
        _indices= []
        # split queues in lane
        for idx in range(0, len(list_vehicle_details['index'])-1):
            if (list_vehicle_details['index'][idx+1] - list_vehicle_details['index'][idx] <= gap_bw_vehicles and
                    finddist(list_vehicle_details['point'][idx+1] ,list_vehicle_details['point'][idx]) < max_dist_gap):
                if list_vehicle_details['point'][idx] not in _queue:
                    _queue.append(list_vehicle_details['point'][idx])
                    _indices.append(list_vehicle_details['index'][idx])
                if list_vehicle_details['point'][idx+1] not in _queue:
                    _queue.append(list_vehicle_details['point'][idx+1])
                    _indices.append(list_vehicle_details['index'][idx+1])
                if (idx == len(list_vehicle_details['index'])-2 ):
                    if (len(_queue) >1):
                        _queue_list.append(_queue)
                        _indices_list.append(_indices)
            else:
                if (len(_queue) > 1):
                    _queue_list.append(_queue)
                    _indices_list.append(_indices)
                _queue = []
                _indices = []
        route_queue_indices.append(_indices_list)
        #  get queue vehicle information and points
        if (len(_queue_list) > 0):
            for qnum in range(0, len(_queue_list)):
                if (len(_queue_list[qnum]) > 1):
                    queue_data['exist'] = True
                    list_axis1_values = [p[axis_vec[0]] for p in laneInformation[route_name][l_num]['lane_points']]
                    list_axis2_values = [p[axis_vec[1]] for p in laneInformation[route_name][l_num]['lane_points']]

                    near_val1 = list_axis1_values[min(range(len(list_axis1_values)), key=lambda i: abs(list_axis1_values[i] - _queue_list[qnum][0][axis_vec[0]]))]
                    near_val2 = list_axis2_values[min(range(len(list_axis2_values)), key=lambda i: abs(list_axis2_values[i] - _queue_list[qnum][-1][axis_vec[1]]))]

                    s_idx = list_axis1_values.index(near_val1)
                    e_idx = list_axis2_values.index(near_val2)

                    if (s_idx> e_idx):
                        temp = e_idx
                        e_idx = s_idx
                        s_idx = temp
                    qdetails = {}
                    q_points = []
                    for _q_line_point in range(s_idx, e_idx+1):
                        q_points.append(laneInformation[route_name][l_num]['lane_points'][_q_line_point])
                    qdetails['points'] = q_points
                    qdetails['n_vehicles'] = _indices_list[qnum]
                    out_queue_info[l_num].append(qdetails)

    #  Observing whether spill_back happening
    if (temporal_data.count(0) == 0):
        spillback_data['ID'] = []
        spillback_data['points'] = []
        previus_ids = []
        for n_frame in range(-1, -num_temporal_frams, -1):
            for l_num in range(0, n_lanes):
                previus_ids.extend(temporal_data[n_frame][route_name][l_num]['ID'])
        for l_num in range(0, n_lanes):
            for current_id in routes_data[route_name][l_num]['ID']:
                if (current_id not in previus_ids):
                    new_id_index = routes_data[route_name][l_num]['ID'].index(current_id)

                    for qind in range(0, len(route_queue_indices[l_num])):
                        if (qind > 0):
                            if new_id_index <= route_queue_indices[l_num][qind][0]  and  new_id_index >= route_queue_indices[l_num][qind-1][0-1]:

                                spill_point = [routes_data[route_name][l_num]['x'][new_id_index], routes_data[route_name][l_num]['y'][new_id_index]]
                                dist_from_start_point = finddist(laneInformation[route_name][l_num]['lane_points'][0], spill_point)
                                if (dist_from_start_point > threshold_spill_to_lane_startpoint):
                                    #spill_back_ids.append(new_id_index)
                                    spillback_data['exist'] = True
                                    spillback_data['ID'].append(routes_data[route_name][l_num]['ID'][new_id_index])
                                    spillback_data['points'].append(spill_point)

                        elif (qind ==0):
                            if new_id_index <= route_queue_indices[l_num][qind][0]:
                                spill_point = [routes_data[route_name][l_num]['x'][new_id_index], routes_data[route_name][l_num]['y'][new_id_index]]
                                dist_from_start_point = finddist(laneInformation[route_name][l_num]['lane_points'][0], spill_point)
                                if (dist_from_start_point > threshold_spill_to_lane_startpoint):
                                    spillback_data['exist'] = True
                                    #spill_back_ids.append(new_id_index)
                                    spillback_data['ID'].append(routes_data[route_name][l_num]['ID'][new_id_index])
                                    spillback_data['points'].append([routes_data[route_name][l_num]['x'][new_id_index], routes_data[route_name][l_num]['y'][new_id_index]])



    queue_data['points'] = out_queue_info
    queue_spillback_data = {}
    queue_spillback_data['queue'] = queue_data
    queue_spillback_data['spillback'] = spillback_data

    return queue_spillback_data

In [17]:
def init_route_info(lane_names):
    '''
    Initializes route information
            Input Parameters:
                    lane_names: Name of the route
            Output Parameters:
                    routes_data: Initial route information
    '''
    routes_data={}
    for route_name in lane_names:
        routes_data[route_name] = []
        for l_num in range(0,3):
            lane_info = {}
            lane_info['ID'] = []
            lane_info['speed'] = []
            lane_info['x'] = []
            lane_info['y'] = []
            routes_data[route_name].append(lane_info)

    return routes_data

In [ ]:
if __name__ == '__main__':
    file_spill_back = open('spillback.txt', 'w')  # Create results file for spill back posistions
    max_length_parameter = 'points' # n_vehicles   input parameter to compute maximum queue length points: trajectory length, n_vehicles: number of vehicles
    file_name = 'competition_dataset.csv'
    data = readdata(file_name)
    traj_props = data['traj_props']
    
    # Method 1
    padd = 1
    routes_accumulater_data = get_accumulator(data, padd, routes_information)

    Lane_information = {}# Init lane information
    route_names = list(routes_information.keys())
    lane_array = {}
    for route_name in route_names:
        lane_array[route_name] = normalizeimg(routes_accumulater_data[route_name][padd:-padd-1, padd:-padd-1] )

    Lane_information['LeofAlexandras_tw_28isOktovriou'] = getLanePoints(lane_array['LeofAlexandras_tw_28isOktovriou'], direction=0)
    Lane_information['OktovriouIs28_tw_LeofAlexandras'] = getLanePoints(lane_array['OktovriouIs28_tw_LeofAlexandras'], direction=225)
    # selecting OktovriouIs28_tw_South start point based on LeofAlexandras_tw_28isOktovriou start point
    Lane_information['OktovriouIs28_tw_South'] = getLanePoints(lane_array['OktovriouIs28_tw_South'], direction=90,
                                                               start_point=Lane_information['LeofAlexandras_tw_28isOktovriou'][0]['lane_points'][0])
    if DISP:
        font = cv2.FONT_HERSHEY_SIMPLEX  # Font style for display text
        fontScale = 0.5  # fontscale for display text
        color = (255, 0, 0)  # font color for display text
        thickness = 2  # font thickness for display text
        legend = np.ones((250, 250, 3), np.uint8) * 255  # vehicle type legend initiolization
        keys = list(obj_types.keys())  # object list
        gap = 3  # gap pixes between each legend parameters
        siz = 15  # size of each legend color box
        for key in range(0, len(keys)):# drow legend on image
            cv2.rectangle(legend, (gap, gap + (key * siz)), ((siz - gap), (siz - gap) + (key * siz)), obj_types[keys[key]]['color'], -1)
            cv2.putText(legend, keys[key], (gap + siz, (siz - gap) + (key * siz)), font, fontScale, obj_types[keys[key]]['color'], 1, cv2.LINE_AA)

        ui_img = np.ones((traj_props['img_height'], traj_props['img_width'] , 3), np.uint8) * 255  # Initiolize image to dispay trajectory
        legend = np.flipud(legend)  # flip virtically legend because world coordinate(Trajectory coordinates) to image coordinates
        ui_img[-legend.shape[0]: , -legend.shape[1]:] = legend  # add legend to initial trajectory image

        # DROW LANE polygon lines
        lane_colors = [(255, 200, 200), (200, 255, 200), (200, 200, 255)]  # lane colors
        for route in route_names:
            for nLane in range(0, len(Lane_information[route])):
                vert = Lane_information[route][nLane]['vertices']  # vertices
                pts = np.array(vert, np.int32)
                cv2.polylines(ui_img, [pts], False, lane_colors[nLane], 1)

    start_time = 0
    end_time = 813
    time_step = 0.04
    num_samples = int((end_time - start_time) / time_step)
    final_sample = int(end_time / time_step)
    vec_time = [int((tim_itr * 0.04 * 100) + 0.5) / 100.0 for tim_itr in range(0, num_samples)]
    object_list = data['object_list']

    temporal_info = [0] * 15  # initiolize buffer for temporal information

    spill_back_data_list = []
    for time_stamp in vec_time:
        if DISP:
            image = np.copy(ui_img)
        routes_data = init_route_info(route_names)
        for obj_id in range(0, len(object_list)):
            if time_stamp in object_list[obj_id]['trajectory']['time']:
                time_index = object_list[obj_id]['trajectory']['time'].index(time_stamp)
                col_range = object_list[obj_id]['trajectory']['x'][time_index]
                row_range = object_list[obj_id]['trajectory']['y'][time_index]
                traj_point = Point(col_range, row_range)  # logitude   lattitude

                if DISP:
                    cv2.rectangle(image, (round(col_range) - 1, round(row_range) - 1), (round(col_range) + 1, round(row_range) + 1), obj_types[object_list[obj_id]['type']]['color'], 1)
                obj_allocate = True
                for route in route_names:
                    if (obj_allocate):
                        for nLane in range(0, len(Lane_information[route])):
                            if(obj_allocate):
                                if (Lane_information[route][nLane]['poly'].contains(traj_point)):
                                    routes_data[route][nLane]['ID'].append(object_list[obj_id]['track_id'] )  #
                                    routes_data[route][nLane]['speed'].append(object_list[obj_id]['trajectory']['speed'][time_index])
                                    routes_data[route][nLane]['x'].append(object_list[obj_id]['trajectory']['x'][time_index])
                                    routes_data[route][nLane]['y'].append(object_list[obj_id]['trajectory']['y'][time_index])
                                    obj_allocate = False
                                    break

        for route_name in route_names:
            # Sort vehicle data in lane according to te position from intersection point
            sort_ids(routes_data, route_name, routes_information[route_name]['direction'])
            # Finding queue points if exists for all routes considering all vehicle speeds are 0 in queue
            _queue_spillback_info = find_queue_info(routes_data,Lane_information,temporal_info ,route_name, routes_information[route_name]['lane_axis'] )

            if _queue_spillback_info['queue']['exist']:
                for l_num in range(0, len(_queue_spillback_info['queue']['points'])):
                    for _queue in _queue_spillback_info['queue']['points'][l_num]:
                        if (len(_queue[max_length_parameter]) > routes_information[route_name]['max_queue']['length']):
                            routes_information[route_name]['max_queue']['length'] = len(_queue['points'])
                            routes_information[route_name]['max_queue']['points'] = _queue['points']
                            routes_information[route_name]['max_queue']['time'] = time_stamp
                            routes_information[route_name]['max_queue']['n_vehicles'] =len(_queue['n_vehicles'])

                        if DISP:
                            pts = np.array(_queue['points'], np.int32)
                            cv2.polylines(image, [pts], False, (255,100,70), 2)

            if _queue_spillback_info['spillback']['exist']:

                for n_spill in range(0,len(_queue_spillback_info['spillback']['points'])):
                    spill_point = _queue_spillback_info['spillback']['points'][n_spill]
                    spill_id = _queue_spillback_info['spillback']['ID'][n_spill]
                    spill_index = object_list[int(spill_id) - 1]['trajectory']['time'].index(time_stamp)

                    spill_data = {}
                    spill_data['Longitude'] = str(object_list[int(spill_id)-1]['trajectory']['lon'][spill_index])
                    spill_data['Latitude'] = str(object_list[int(spill_id) - 1]['trajectory']['lon'][spill_index])
                    spill_data['Object_ID'] = spill_id
                    spill_data['Time'] = time_stamp
                    spill_data['SpillBack_Route_area'] = route_name
                    spill_back_data_list.append(spill_data)

                    if DISP:
                        spill_point = _queue_spillback_info['spillback']['points'][n_spill]
                        cv2.circle(image, (int(spill_point[0]), int(spill_point[1])), 4, (0,0,255), 2)


        #exist1, slippback_data = find_spillback_info(temporal_info)
        temporal_info.pop(0)
        temporal_info.append(routes_data)
        if DISP:
            image = np.flipud(image)
            image = np.array(image)
            cv2.putText(image, 'Time: ' + str(time_stamp), (image.shape[1] - 400, 30), font, 0.7, (255, 100, 100), 1, cv2.LINE_AA)
            cv2.imshow("UAS Trajectory View",image)
            cv2.waitKey(1)
            # Make a list for temporal information


    with open('results.csv', mode='w') as results_file:

        writer_q = csv.DictWriter(results_file, fieldnames = ['Route_name','Time_stamp','Maximum_queue_length_points','Maximum_queue_length_meters', 'Num_vehicles','Coordinates'])
        writer_q.writeheader()
        for route_name in route_names:
            if (routes_information[route_name]['max_queue']['length'] > 0):
                coordinates = []
                for pnum in range(0, len(routes_information[route_name]['max_queue']['points'])):
                    coordinates.append((((routes_information[route_name]['max_queue']['points'][pnum][0] / float(data['traj_props']['scale_trajectory'])) + traj_props['lon_min']),
                                          ((routes_information[route_name]['max_queue']['points'][pnum][1] / float(data['traj_props']['scale_trajectory'])) + traj_props['lat_min'])))

                distance = finddist(coordinates[0], coordinates[-1])
                dist_m = distance * 112 * 1000  # world coordinates to meters
                writer_q.writerow({'Route_name':route_name,
                                   'Time_stamp': str(routes_information[route_name]['max_queue']['time']),
                                   'Maximum_queue_length_points':routes_information[route_name]['max_queue']['length'],
                                   'Maximum_queue_length_meters' :dist_m,
                                   'Num_vehicles':routes_information[route_name]['max_queue']['n_vehicles'],
                                   'Coordinates': coordinates}
                                  )

        if (len(spill_back_data_list) > 0 ):
            spill_back_fields = list(spill_back_data_list[0].keys())
            writer = csv.DictWriter(results_file, fieldnames=spill_back_fields)
            writer.writeheader()
            for spill in spill_back_data_list:
                writer.writerow(spill)